In [1]:
import xsimlab as xs
import pgljupyter as pj
import vmlab

In [2]:
from vmlab.models import vmango

### Define 25 'interpretation_steps' evenly distributed between start and end date

In [4]:
setup = vmlab.create_setup(
    model=vmango,
    start_date='2003-06-01',
    end_date='2005-06-01',
    setup_toml='vmango.toml',
    input_vars={
        'geometry__interpretation_steps': 25
    },
    output_vars=None
)

### Setup a hook to retrieve and plot trees on a 5x5 plane during the simulation 

In [5]:
def plot_orchard():
    from math import ceil, sqrt, floor
    from collections import deque
    positions = deque()
    sw = None
    @xs.runtime_hook(stage='run_step')
    def hook(model, context, state):
        nonlocal sw, positions
        if context['step'] == 0:
            length = int(state[('geometry', 'interpretation_steps')])
            cell = 3.5
            rows = cols = ceil(sqrt(length))
            size = rows * cell
            start = -size / 2 + cell / 2
            for i in range(length):
                row = floor(i / rows)
                col = (i - row * cols)
                x = row * cell + start
                y = col * cell + start
                positions.append((x, y, 0))
            sw = pj.SceneWidget(size_world=cell*rows, size_display=(600, 400))
            display(sw)
        scene = state[('geometry', 'scene')]
        if scene is not None:
            sw.add(scene, position=positions.popleft(), scale=1/100)
    return hook

### Run vmango

In [6]:
_ = vmlab.run(setup, vmango, hooks=[plot_orchard()], progress=False)

SceneWidget(scenes=[{'id': 'zol5CCasz90wGqB8SK1cy9eqs', 'data': b'x\xdaSLrw\xf5\xf7e`Pp\xe0\xe5RPVVd\x00\x020\…